# NLP Project: Sentiment Analysis

-------------------------------------------------------------------------------------
Nom: Rouis Ibrahim

Goupe: 5GL B

Sous Groupe 2

-------------------------------------------------------------------------------------

In [24]:
import pandas as pd
import requests
import textwrap

## Define model
The model we are running is [granite3.3:2b](https://ollama.com/library/granite3.3) running on localhost thanks to [Ollama](https://ollama.com/)

Ollama is the easiest way to get up and running with large language models such as granite3.3, gpt-oss, Gemma 3, DeepSeek-R1, Qwen3 and more.

The IBM Granite 2B models is 128K context length language models that have been fine-tuned for improved reasoning and instruction-following capabilities.

Granite 3.3 is a decoder‑only transformer language model

In [25]:
OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "granite3.3:cpu"

## Load reviews.csv
The reviews.csv file is a collection of reviews fetched by me from [yelp](https://www.yelp.com/)
It contains two columns: text and sentiment.

In [26]:
df = pd.read_csv("reviews.csv", encoding="cp1252")
df.head()


,text,sentiment
0,"The food was incredible. Not only did Nomi, ou...",Positive
1,This was my first time eating at Fog Harbor Fi...,Positive
2,We had reservations here on a Saturday night w...,Positive
3,While the setting is magical and its decked ou...,Negative
4,Plenty of sourdough rolls. Near the end of the...,Positive


## Define sentiment analysis functions

In [27]:
# Function to analyze sentiment using the local LLM
def analyze_sentiment(text) -> str:
    try:
        prompt = f"Respond only with one word: Positive or Negative\n" f"Text: {text}"
        payload = {"model": MODEL_NAME, "prompt": prompt, "stream": False}
        response = requests.post(OLLAMA_URL, json=payload)
        response.raise_for_status()
        result = response.json()
        # Extract the generated sentiment
        return result.get("response", "").strip()
    except requests.RequestException as e:
        print(f"Request error: {e}")
        return ""

In [28]:
# Function to verify the model's predictions against the dataset
def verify_model(df: pd.DataFrame):
    correct = 0
    total = len(df)
    evaluated = 0
    for row in df.itertuples(index=False):
        text = row.text
        sentiment = row.sentiment 
        prediction = analyze_sentiment(text)
        if not prediction:
            continue  # skip unknown/failed responses
        evaluated += 1
        if prediction == sentiment:
            correct += 1
        if evaluated % 10 == 0:
            print(f"Progress: {evaluated}/{total} evaluated...")
    accuracy = (correct / evaluated) if evaluated > 0 else 0.0
    return accuracy, evaluated, total

## Try analyze sentiment on a review

In [29]:
first_review = df.iloc[2]
review_text = first_review['text']
sentiment = first_review['sentiment']
predicted = analyze_sentiment(review_text)
print("-----------------------------------------------------------------")
print("Review:", textwrap.fill(review_text, width=50, replace_whitespace=False))
print("-----------------------------------------------------------------")
print()
print("Correct sentiment:", sentiment)
print("Predicted sentiment:", predicted)

-----------------------------------------------------------------
Review: We had reservations here on a Saturday night which
I definitely recommend since the restaurant was
pretty full. The interior is really nice with a
view of the bay and an upscale but comfortable
environment. The service here is pretty decent, we
were checked on multiple times and the server was
friendly too.

The Caesar salad was my least
favorite thing, it lacked dressing and croutons.
Everything else here was really good though, I had
the seasonal miso salmon which is the best salmon
I've ever had! The miso dressing wasn't too sweet
or overpowering and paired perfectly with the bok
choy. The lobster roll was nice and buttery. The
substituted side of sautéed vegetables was
garlicky and satisfying. The clam chowder was also
a 5/5 for me. Overall this is a nice seafood
dinner spot with a view, and a nice dinner choice
on the pier.
-----------------------------------------------------------------

Correct sentiment:

## Verify model

In [30]:
print("Verifying model...")
accuracy, evaluated, total = verify_model(df)
print(f"Evaluated: {evaluated}/{total}")
print(f"Model accuracy: {accuracy * 100:.2f}%")

Verifying model...
Progress: 10/11 evaluated...
Evaluated: 11/11
Model accuracy: 100.00%
